In [1]:
import os
import pandas as pd

from pprint import pprint

from sklearn.metrics import accuracy_score

from skllm import MultiLabelZeroShotGPTClassifier
from skllm.config import SKLLMConfig

In [2]:
#See notes in INSTALL.md for how to set this, DO NOT HARD CODE YOUR API KEY HERE, 
# if your repository is public, then someone will steal your API key and make you pay for their shit 
SKLLMConfig.set_openai_key(os.environ.get('OPENAI_API_KEY'))

# Collect a Dataset

In [3]:
# Load the Reddit life tips dataset
data = pd.read_csv('data/Master.csv')

# Understand Your Data

In [4]:
data.head()

,playerID,coachID,hofID,firstName,lastName,nameNote,nameGiven,nameNick,height,weight,...,birthDay,birthCountry,birthState,birthCity,deathYear,deathMon,deathDay,deathCountry,deathState,deathCity
0,aaltoan01,NaN,NaN,Antti,Aalto,NaN,Antti,NaN,73.0,210.0,...,4.0,Finland,NaN,Lappeenranta,NaN,NaN,NaN,NaN,NaN,NaN
1,abbeybr01,NaN,NaN,Bruce,Abbey,NaN,Bruce,NaN,73.0,185.0,...,18.0,Canada,ON,Toronto,NaN,NaN,NaN,NaN,NaN,NaN
2,abbotge01,NaN,NaN,George,Abbott,NaN,George Henry,Preacher,67.0,153.0,...,3.0,Canada,ON,Synenham,NaN,NaN,NaN,NaN,NaN,NaN
3,abbotre01,NaN,NaN,Reg,Abbott,NaN,Reginald Stewart,NaN,71.0,164.0,...,4.0,Canada,MB,Winnipeg,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelju01,NaN,NaN,Justin,Abdelkader,NaN,NaN,NaN,73.0,195.0,...,25.0,USA,MI,Muskegon,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.describe()

,height,weight,ihdbID,firstNHL,lastNHL,firstWHA,lastWHA,birthYear,birthMon,birthDay,deathYear,deathMon,deathDay
count,7334.000000,7336.000000,7125.000000,6851.000000,6851.000000,903.000000,903.000000,7543.000000,7506.000000,7504.000000,1320.000000,1173.000000,1168.000000
mean,71.904418,189.503953,18159.395930,1979.898555,1985.173551,1974.256921,1975.746401,1955.170091,6.056088,15.605144,2547.587121,6.670929,15.244863
std,2.328650,18.341611,22852.778006,24.150092,24.359206,2.123774,1.947460,26.379075,3.434267,8.917103,2052.311223,3.484605,8.601029
min,63.000000,125.000000,45.000000,1917.000000,1917.000000,1972.000000,1972.000000,1841.000000,1.000000,1.000000,1907.000000,1.000000,1.000000
25%,70.000000,178.000000,3339.000000,1968.000000,1974.000000,1972.000000,1974.000000,1943.000000,3.000000,8.000000,1970.000000,4.000000,8.000000
50%,72.000000,190.000000,7500.000000,1985.000000,1991.000000,1974.000000,1976.000000,1961.000000,6.000000,15.000000,1989.000000,7.000000,15.000000
75%,73.000000,200.000000,23463.000000,1999.000000,2006.000000,1976.000000,1978.000000,1975.000000,9.000000,23.000000,2002.000000,10.000000,23.000000
max,81.000000,265.000000,118280.000000,2011.000000,2011.000000,1978.000000,1978.000000,1993.000000,12.000000,31.000000,9999.000000,12.000000,31.000000


In [6]:
X = []

for index, row in data.iterrows():
    player_data = row

    
    profile = f"Player Profile:\n"
    
    if not pd.isna(player_data['nameGiven']):
        profile += f"Name: {player_data['nameGiven']} \n"
    
    
    if not pd.isna(player_data['height']):
        profile += f"Height: {player_data['height']} inches\n"
    
    if not pd.isna(player_data['weight']):
        profile += f"Weight: {player_data['weight']} pounds\n"
    
    if not pd.isna(player_data['birthYear']):
        profile += f"Birth Year: {int(player_data['birthYear'])}\n"
    
    if not pd.isna(player_data['birthMon']) and not pd.isna(player_data['birthDay']):
        profile += f"Date of Birth: {int(player_data['birthMon'])}/{int(player_data['birthDay'])}\n"
    
    if not pd.isna(player_data['firstNHL']) and not pd.isna(player_data['lastNHL']):
        profile += f"NHL Career: {int(player_data['firstNHL'])}-{int(player_data['lastNHL'])}\n"
    
    if not pd.isna(player_data['firstWHA']) and not pd.isna(player_data['lastWHA']):
        profile += f"WHA Career: {int(player_data['firstWHA'])}-{int(player_data['lastWHA'])}\n"
    
    if not pd.isna(player_data['deathYear']) and not pd.isna(player_data['deathMon']) and not pd.isna(player_data['deathDay']):
        profile += f"Date of Death: {int(player_data['deathMon'])}/{int(player_data['deathDay'])}/{int(player_data['deathYear'])}\n"
    
    X.append(profile)

In [7]:
X

['Player Profile:\nName: Antti \nHeight: 73.0 inches\nWeight: 210.0 pounds\nBirth Year: 1975\nDate of Birth: 3/4\nNHL Career: 1997-2000\n',
 'Player Profile:\nName: Bruce \nHeight: 73.0 inches\nWeight: 185.0 pounds\nBirth Year: 1951\nDate of Birth: 8/18\nWHA Career: 1975-1975\n',
 'Player Profile:\nName: George Henry \nHeight: 67.0 inches\nWeight: 153.0 pounds\nBirth Year: 1911\nDate of Birth: 8/3\nNHL Career: 1943-1943\n',
 'Player Profile:\nName: Reginald Stewart \nHeight: 71.0 inches\nWeight: 164.0 pounds\nBirth Year: 1930\nDate of Birth: 2/4\nNHL Career: 1952-1952\n',
 'Player Profile:\nHeight: 73.0 inches\nWeight: 195.0 pounds\nBirth Year: 1987\nDate of Birth: 2/25\nNHL Career: 2007-2011\n',
 'Player Profile:\nName: Clarence John \nHeight: 73.0 inches\nWeight: 225.0 pounds\nBirth Year: 1900\nDate of Birth: 5/28\nNHL Career: 1926-1933\nDate of Death: 8/1/1964\n',
 'Player Profile:\nName: Gerald Scott \nHeight: 74.0 inches\nWeight: 168.0 pounds\nBirth Year: 1944\nDate of Birth: 12/2

In [8]:
X_ten = X[:10]
X_ten

['Player Profile:\nName: Antti \nHeight: 73.0 inches\nWeight: 210.0 pounds\nBirth Year: 1975\nDate of Birth: 3/4\nNHL Career: 1997-2000\n',
 'Player Profile:\nName: Bruce \nHeight: 73.0 inches\nWeight: 185.0 pounds\nBirth Year: 1951\nDate of Birth: 8/18\nWHA Career: 1975-1975\n',
 'Player Profile:\nName: George Henry \nHeight: 67.0 inches\nWeight: 153.0 pounds\nBirth Year: 1911\nDate of Birth: 8/3\nNHL Career: 1943-1943\n',
 'Player Profile:\nName: Reginald Stewart \nHeight: 71.0 inches\nWeight: 164.0 pounds\nBirth Year: 1930\nDate of Birth: 2/4\nNHL Career: 1952-1952\n',
 'Player Profile:\nHeight: 73.0 inches\nWeight: 195.0 pounds\nBirth Year: 1987\nDate of Birth: 2/25\nNHL Career: 2007-2011\n',
 'Player Profile:\nName: Clarence John \nHeight: 73.0 inches\nWeight: 225.0 pounds\nBirth Year: 1900\nDate of Birth: 5/28\nNHL Career: 1926-1933\nDate of Death: 8/1/1964\n',
 'Player Profile:\nName: Gerald Scott \nHeight: 74.0 inches\nWeight: 168.0 pounds\nBirth Year: 1944\nDate of Birth: 12/2

# "Develop" a Model aka just use OpenAI's API

In [11]:
# Define candidate labels
candidate_labels = [
    "Center",
    "Wing",
    "Defense",
    "Goalie"
    
]


# Create and fit the classifier
clf = MultiLabelZeroShotGPTClassifier(max_labels=2) 
clf.fit(None, [candidate_labels])

MultiLabelZeroShotGPTClassifier(max_labels=2)

In [13]:
# Predict the labels
labels = clf.predict(X_ten)

100%|███████████████████████████████████████████| 10/10 [00:16<00:00,  1.66s/it]


In [14]:
print(labels)

[['Defense'], ['Defense'], ['Center'], ['Center'], ['Defense'], ['Defense'], ['Center'], ['Center'], ['Defense'], ['Defense']]


In [16]:
# Add labels to the dataset and save
# data['ChatGPTLabel'] = labels
# data.to_csv('data/Master.csv', index=False)
zipped = zip(X_ten, labels)
data = pd.DataFrame(zipped, columns = ["profile", "position"])
print(data)
data.to_csv('data/nhloutput.csv', index=False)

                                             profile   position
0  Player Profile:\nName: Antti \nHeight: 73.0 in...  [Defense]
1  Player Profile:\nName: Bruce \nHeight: 73.0 in...  [Defense]
2  Player Profile:\nName: George Henry \nHeight: ...   [Center]
3  Player Profile:\nName: Reginald Stewart \nHeig...   [Center]
4  Player Profile:\nHeight: 73.0 inches\nWeight: ...  [Defense]
5  Player Profile:\nName: Clarence John \nHeight:...  [Defense]
6  Player Profile:\nName: Gerald Scott \nHeight: ...   [Center]
7  Player Profile:\nName: Sidney Gerald \nHeight:...   [Center]
8  Player Profile:\nName: Dennis Harvey \nHeight:...  [Defense]
9  Player Profile:\nHeight: 74.0 inches\nWeight: ...  [Defense]


# Choose a measure of success, Choose an evaluation protocol / evaluate

In [17]:
data

,profile,position
0,Player Profile:\nName: Antti \nHeight: 73.0 in...,[Defense]
1,Player Profile:\nName: Bruce \nHeight: 73.0 in...,[Defense]
2,Player Profile:\nName: George Henry \nHeight: ...,[Center]
3,Player Profile:\nName: Reginald Stewart \nHeig...,[Center]
4,Player Profile:\nHeight: 73.0 inches\nWeight: ...,[Defense]
5,Player Profile:\nName: Clarence John \nHeight:...,[Defense]
6,Player Profile:\nName: Gerald Scott \nHeight: ...,[Center]
7,Player Profile:\nName: Sidney Gerald \nHeight:...,[Center]
8,Player Profile:\nName: Dennis Harvey \nHeight:...,[Defense]
9,Player Profile:\nHeight: 74.0 inches\nWeight: ...,[Defense]


In [ ]:
accuracy = accuracy_score(data[["HumanLabel"]], data[["ChatGPTLabel"]])
print(accuracy)

# Skipped Steps
* Beat a baseline
* Overfit, regularize and tune
* Communicate with stakeholders
* Ship an inference model
* Monitor and maintain